In [1]:
import sys
import pandas as pd
import numpy as np
import plotly.express as px
import xgboost as xgb

from sklearn import preprocessing
from sklearn.model_selection import train_test_split



In [262]:
shared_drive = r"G:\.shortcut-targets-by-id\184tVjsIO-GAjbkSakwDbEZ40M5mPpgu4\Capstone\cleaned_data"
google_drive = r"G:\My Drive\Spring_2022\CS554\Project\data"
drive = r"D:\Users\yiboz\Programming\Github\CS554\data"
articles = pd.read_csv(shared_drive+r"\articles_clean.csv", dtype={'article_id':str})
customers = pd.read_csv(shared_drive+r"\customers_clean.csv", dtype={'customer_id':str})
transactions = pd.read_csv(shared_drive+r"\transactions_train.csv", dtype={'article_id':str, 'customer_id':str})
sample = pd.read_csv(shared_drive+r"\sample_submission.csv")

In [252]:
sample.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0706016001 0706016002 0372860001 0610776002 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0706016001 0706016002 0372860001 0610776002 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0706016001 0706016002 0372860001 0610776002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0706016001 0706016002 0372860001 0610776002 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0706016001 0706016002 0372860001 0610776002 07...


In [256]:
sample['prediction'][0]

'0706016001 0706016002 0372860001 0610776002 0759871002 0464297007 0372860002 0610776001 0399223001 0706016003 0720125001 0156231001'

NOTE: Look back again on how the Movie reccomendation rating thing worked, might need 100k+ columns for 1.3mil customers (looking at possibly 10BIL+ rows), regression rating on each. Definetely need pyspark. Using ratings to rank their top 12 preferences 


Questions: 
    
    1. How to use a list of article_ids as outputs? 
    2. How to get a list of predictions of article_ids? 
    
    3. How are we supposed to use the transaction date values? Do we want to use it? 
    4. What are the significant columns in the articles table we wanted to use? Need to use Chi-square to test. 
    5. Articles have a high cardinality and I need to reduce their dimension? What is the best way to encode the article categories? 
        - https://pbpython.com/categorical-encoding.html 
    6. What is the type of regressor I want to use for my ranking data? 


The Plan: Need to use regression on ratings of all articles? (just use articles from Transactions)  
    
    Ratings: Get value_counts of each article_id bought and normalize by taking (Max_frequency - Min_frequency) (need pivot table). Answers questions 1 and 2. 
    
    Encoding: Categorical vs Binary due to High Cardinality of article_id 
    
    Buckets: Group customers into buckets of close age brackets (possibly based on the distribution) and other characteristics 
    
    Reduction of Articles: drop the bottom percent of least popular articles
    
    Reduction of Customers: use only a small percentage of customers to train/test the data, then get recommendations on everyone else 
    
    Regressor: Try 'reg:squarederror', need to look into rank:pairwise and rank:map
    
    Evaluation: default set by objective function, ex. 'rmse'. 

TODO: figure out the most significant articles columns, all categorical so use Chi-squared test 

X-values (did we ever figure out the most significant columns for all of the articles?): ['customer_id', 'age', 'FN', 'Active']

    Need to seperate customer and article characteristics for a hybrid approach, possibly combine afterwards 

Y-values: ratings on all articles 

Observations: 

    1. The data is horribly imbalanced, with thousands of users having only purchased one item 

In [3]:
# index group is the smallest at 5 different categories 
mini = sys.maxsize
col = ''
for i in articles.columns: 
    n = articles[i].unique().size
    if mini > n: 
        mini = n
        col = i
print("col: "+col+" size: "+ str(mini))


col: index_group_no size: 5


In [293]:
customer_attributes = ['age', 'FN', 'Active', 'customer_id']
article_attributes = ['price', 'index_group_name', 'article_id']
target = "rating"

In [265]:
df = transactions.sample(frac=0.01)
df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
14603869,2019-07-27,0e5b3379c1c3fc68cd905070104f6aab53c493299e893a...,0531615001,0.011847,2
4901204,2019-01-12,74bd41da228573d992e2e3cf26814e7817667b4d09d3cd...,0704629002,0.047441,2
8411446,2019-04-06,b1eda66468fb21b6f29022c341a15c1c2dfc080ba0aae3...,0732429001,0.094898,2
31739575,2020-09-21,8156c342e1c0c61eb1e603a74fb7788ef32204538c0ae4...,0874754003,0.033881,2
24837418,2020-04-20,0e8e816a0c019b94a174776e7d6d30520d7a140b2204d8...,0837306003,0.033881,2


In [346]:
# returns the number of features necessary for binary encoding 
# Input: cardinality of feature
# Output: float of the number of columns needed to represent it in binary encoding 
def binary_features(n): 
    return np.ceil(np.log(n+1)/np.log(2))

# Get purchase counts of each article for each customer 
# Input: df=base dataframe 
# Output: dataframe with a column for those purchase counts 
def purchase_count(df, customer="customer_id", projectPath=False): # from Xingeng 
    customArticlePair = df[[customer, "article_id"]].copy()
    # customArticlePair.loc[:,"rating"] = pd.Series([1 for x in range(len(customArticlePair))]) # Add a column of purchase count
    customArticlePair["count"] = 1
    # customArticlePair.to_csv("/content/gdrive/MyDrive/Capstone/co_filter/customer_article_pair.csv",index=False)
    countGroup = customArticlePair.groupby([customer, "article_id"]).count() # count purchase numbers
    
    if projectPath != False: 
        countGroup.to_csv(f"{projectPath}/customer_article_count.csv") # Save Counts to csv file
    return countGroup.reset_index()

# not used, reference 
def pivot_norm_purchase_count(df, customer="customer_id"): # from Xingeng 
    df_matrix = pd.pivot_table(df, values='rating', index=customer, columns='article_id')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    # print(d.head())
    # d.index.names = ['scaled_purchase_freq']
    # return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
    
    return d

# get ratings of each article for each customer by taking (article count for customer) / (max count for customer)
# Input: df=dataframe with purchase counts 
# Output: df with 'ratings' column 
def norm_ratings(df): 
    tempmax = df.groupby('customer_id')['count'].max().rename("temp_max") # get max for each customer_id 
    temp = df.merge(tempmax, how='left', on='customer_id') # merge the data together 
    temp['rating'] = temp['count'] / temp['temp_max'] # get true rating 
    return temp.drop(['count', 'temp_max' ], axis=1)

# map categorical variables to numeric or binary, returns a mapped df and decoder dictionary 
# Input: df=base dataframe, col=column to create a mapping for, to_bin=map to binary 
# Output: df with a 'col_mapped' column, decoder dictionary for the mapping 
def mapper(df, col, to_bin=False): 
    n = df[col].unique().size
    mp = dict.fromkeys(range(n*2))
    de = dict.fromkeys(range(n*2))
    j = 1 
    
    if to_bin:
        b = str(int(binary_features(n)))
        f = "{:0>"+b+"b}"
    
    for i in df[col].unique(): 
        if to_bin: 
            mp[i] = f.format(j)
            de[f.format(j)] = i
        else: 
            mp[i] = j
        
        j += 1   
    df[col+'_mapped'] = df[col].apply(lambda x: mp[x] )
    return df, de


# creating binary encoding for a column  
# Input: df=base dataframe, col=column to encode into binary 
# Output: dataframe with n columns for binary encoding of a categorical variable, decoder dictionary 
def binary_encoder(df, col): 
    encoding = df[col].to_frame().copy() 
    mapped, de = mapper(encoding, col, to_bin=True)
    
    f = lambda x: col+'_b_{}'.format(x + 1)
    binned = pd.DataFrame(mapped.pop(col+'_mapped').apply(list).values.tolist()).apply(pd.to_numeric).rename(columns=f)
    return mapped.join(binned), de
    

# Raw to train, preps all data for xgboosting 
# Input: df=base dataframe, att_df=datframe with the user or article attributes, atts=list of attributes, 
# att_id=column name of attribute, cats=name of categorical variable to encode, goal=name of target column 
# Output: Dataframe of attributes, dataframe of ratings, decoder dictionary 
def training_data_prep(df, att_df, atts, att_id, cats, goal): 
    purchases = purchase_count(df)
    normed = norm_ratings(purchases)
    cust = att_df[atts].copy().set_index(att_id)
    merged_df = normed.merge(cust, how='left', on=att_id).drop(att_id, axis=1)
    
    encoded, de = binary_encoder(merged_df[cats].to_frame().copy(), cats) 
    
    merged_df = merged_df.join(encoded.drop(cats, axis=1))
    y_all = merged_df[goal].to_frame()
    X_all = merged_df.drop([goal, cats], axis=1)
    
    return X_all, y_all, de
    #return merged_df

# Given prediction test x and test y, get root mean square prediction 
# Input: x=testing attributes, y=testing ratings 
# Output: root mean squared error between x and y 
def test_pred_rms(x, y): 
    pred = pd.DataFrame(x, columns=['pred'])
    
    y = y.reset_index()
    del y['index']
    
    result = pd.concat([pred, y], ignore_index=True, axis=1, join='inner')
    
    f = lambda x: 'p_{}'.format(x + 1)
    result = result.rename(columns=f)
    result['diff'] = result['p_1'] - result['p_2']
    result['square'] = result['diff'] ** 2 

    return np.sqrt(result['square'].mean()) 

# prepares the data for a user ID for prediction in the finished model 
# Input: user=string of customer_id, encoded_articles=pre-built dataframe of all unique articles in the set
# Output: customer attributes with encoded, dataframe of customer and article IDs for decoding 
def test_user_prep(user, encoded_articles):
    cust = encoded_articles.copy()
    cust['customer_id'] = user
    
    ids = cust[['customer_id', 'article_id']].copy()
    
    cust = cust.merge(customers[customer_attributes].set_index('customer_id'), on='customer_id', how='left')
    cust = cust[['age', 'FN', 'Active']+[c for c in cust if c not in ['age', 'FN', 'Active']]]
    
    return cust.drop(['customer_id', 'article_id'], axis=1), ids

# Turn prediction values into a recommendation for a user 
# Input: df=predicted rating scores, ids=decoding IDs for customers and articles, n=top number of ratings to get 
# Output: dataframe with recommendation in submission form 
def get_user_rec(df, ids, n): 
    pred = pd.DataFrame(df, columns=['rating']).copy()
    top12 = pred.sort_values(["rating"], ascending=False).head(n)
    joined = ids.reset_index().merge(top12, left_on='index', right_on=top12.index, how='inner')
    rec = pd.DataFrame(joined.groupby(by='customer_id').apply(agg_articles)).reset_index().rename(columns={0:'prediction'})
    
    return rec 
    
# aggregate top articles into 
# Input: df=dataframe of top articles 
# Output: string of those articles in submission form 
def agg_articles(df): 
    return ' '.join(df['article_id'].tolist())   
    

In [268]:
X_all, y_all, de = data_prep(df, customers, customer_attributes, "customer_id", 'article_id', target)
X_all.head()

,age,FN,Active,article_id_b_1,article_id_b_2,article_id_b_3,article_id_b_4,article_id_b_5,article_id_b_6,article_id_b_7,article_id_b_8,article_id_b_9,article_id_b_10,article_id_b_11,article_id_b_12,article_id_b_13,article_id_b_14,article_id_b_15,article_id_b_16
0,24.0,0.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,52.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,32.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
3,56.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,56.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1


In [269]:
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.33, random_state=42)

In [270]:
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", n_jobs=13,random_state=42,n_estimators=100)

In [271]:
xgb_model.fit(X_train, y_train, 
              verbose=True, 
              eval_metric = 'rmse')

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=13,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [272]:
pred1 = xgb_model.predict(X_test)

In [273]:
test_pred_rms(pred1, y_test)

0.024915172550973627

NEXT STEP: 
1. Process all users for their ratings on all articles of clothing 
2. Get articles with the top 12 ratings, decode the article info 
3. Create a submission, concat the articles into a string 



In [277]:
arts = pd.DataFrame(df['article_id'].unique(), columns=['article_id'])

,article_id
0,0531615001
1,0704629002
2,0732429001
3,0874754003
4,0837306003
...,...
53590,0318035017
53591,0747390001
53592,0751429001
53593,0697287002


In [280]:
encoded_articles, dec = binary_encoder(arts, 'article_id')
encoded_articles

,article_id,article_id_b_1,article_id_b_2,article_id_b_3,article_id_b_4,article_id_b_5,article_id_b_6,article_id_b_7,article_id_b_8,article_id_b_9,article_id_b_10,article_id_b_11,article_id_b_12,article_id_b_13,article_id_b_14,article_id_b_15,article_id_b_16
0,0531615001,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0704629002,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0732429001,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
3,0874754003,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0837306003,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53590,0318035017,1,1,0,1,0,0,0,1,0,1,0,1,0,1,1,1
53591,0747390001,1,1,0,1,0,0,0,1,0,1,0,1,1,0,0,0
53592,0751429001,1,1,0,1,0,0,0,1,0,1,0,1,1,0,0,1
53593,0697287002,1,1,0,1,0,0,0,1,0,1,0,1,1,0,1,0


In [284]:
cust = sample['customer_id'][0]
cust

'00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657'

In [311]:
prep1, ids = test_user_prep(cust)
prep1.head()

,age,FN,Active,article_id_b_1,article_id_b_2,article_id_b_3,article_id_b_4,article_id_b_5,article_id_b_6,article_id_b_7,article_id_b_8,article_id_b_9,article_id_b_10,article_id_b_11,article_id_b_12,article_id_b_13,article_id_b_14,article_id_b_15,article_id_b_16
0,49.0,0.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,49.0,0.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,49.0,0.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
3,49.0,0.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,49.0,0.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1


In [343]:
pred3 = xgb_model.predict(prep1)

In [344]:
rec = get_user_rec(pred3, ids,12)
rec

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0784415002 0570319014 0898243001 0617249034 06...


In [314]:
top12 = pred3.sort_values(["rating"], ascending=False).head(12)
top12

,rating
45900,1.003976
40434,1.003756
33018,1.003754
34034,1.003716
48134,1.003658
33017,1.003646
48840,1.003622
48836,1.003609
48835,1.003583
50897,1.003568


In [325]:
test = ids.reset_index().merge(top12, left_on='index', right_on=top12.index, how='inner')

In [330]:
test

,index,customer_id,article_id,rating
0,33017,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0784415002,1.003646
1,33018,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0570319014,1.003754
2,34034,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0898243001,1.003716
3,40434,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0617249034,1.003756
4,45390,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0664779009,1.003548
5,45900,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0835704009,1.003976
6,45932,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0672016010,1.003560
7,48134,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0924161001,1.003658
8,48835,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0636784001,1.003583
9,48836,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0541418003,1.003609


In [331]:
agg_articles(test)

'0784415002 0570319014 0898243001 0617249034 0664779009 0835704009 0672016010 0924161001 0636784001 0541418003 0722832001 0779612005'

In [341]:
rec = pd.DataFrame(test.groupby(by='customer_id').apply(agg_articles)).reset_index().rename(columns={0:'prediction'})
rec

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0784415002 0570319014 0898243001 0617249034 06...


In [11]:
a = articles['article_id'].count()
b = transactions['article_id'].unique().size
a - b

In [14]:
transactions['customer_id'].value_counts()

be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee985513d9e8e53c6d91b    1895
b4db5e5259234574edfff958e170fe3a5e13b6f146752ca066abca3c156acc71    1441
49beaacac0c7801c2ce2d189efe525fe80b5d37e46ed05b50a4cd88e34d0748f    1364
a65f77281a528bf5c1e9f270141d601d116e1df33bf9df512f495ee06647a9cc    1361
cd04ec2726dd58a8c753e0d6423e57716fd9ebcf2f14ed6012e7e5bea016b4d6    1237
                                                                    ... 
63b70b71291668f0a63ade8e321fb3eccb80eba164f2087dad471de065f18e1f       1
950b172c36d169bf427545991fe66371f21a085799b44780fdcb2da6a3091613       1
7c284f13f4af9d6a53f97279381638ed0cb7afaa4fd4f3eaadc21993ea45fc69       1
62d49d0ae11a4f65fa31e354cb87f6b557ebec648e0e5e71435d2dd190d1ccc4       1
268eaa31a07d6f2f4f060bfcf32a660f3ea3dbb21ef14cd09fc2545f4e0b5c10       1
Name: customer_id, Length: 1362281, dtype: int64

In [15]:
big = 'be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee985513d9e8e53c6d91b'

In [25]:
u = transactions.loc[(transactions['customer_id'] == big)]['article_id'].count()
u

In [21]:
transactions.loc[(transactions['customer_id'] == big)]['article_id'].value_counts()

253448001    8
826500008    7
668956001    7
828934001    6
777099001    6
            ..
562245089    1
680262004    1
807064003    1
798579002    1
879291001    1
Name: article_id, Length: 1346, dtype: int64

In [8]:
df.article_id.value_counts()

706016001    480
706016002    404
372860001    321
610776002    286
372860002    255
            ... 
614423001      1
613666002      1
776087001      1
636868001      1
730683013      1
Name: article_id, Length: 53537, dtype: int64

In [10]:
df.customer_id.value_counts()

be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee985513d9e8e53c6d91b    23
67931690bdf18d2e328854ae772cd5ce2505fdc11164693998b13e706db0bb56    21
a65f77281a528bf5c1e9f270141d601d116e1df33bf9df512f495ee06647a9cc    18
a76cf5ea515d09f22b7fe3e8ea3c1944316bd6264a90e26cef126242ef3c5e11    17
03d0011487606c37c1b1ed147fc72f285a50c05f00b9712e0fc3da400c864296    17
                                                                    ..
328476e33eb9a28207809832497cef3e7ea0e0d5b5f7f302c51fd24d63ae6653     1
6b6c4268ceba96e1858768dc56acb4b44c0be24b96aa50df0d90785629301374     1
f9a9fd743222c81e259912bc45900dff506f18168378f1bf80f9ffc4fe27ba4a     1
019841f5cdfcb8cffa470612d2ceb690d2b5374e728657edb8e437efcd43a000     1
7cd8624699da578691cd501bf0e9d04ed67b50f18cbbf2207722678c180f1e33     1
Name: customer_id, Length: 229955, dtype: int64